# Checks

In [28]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU device name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")


GPU is available!
GPU device name: Tesla V100-PCIE-32GB


In [29]:
import os

def is_slurm_job():
    # Check if common SLURM environment variables are set
    return 'SLURM_JOB_ID' in os.environ or 'SLURM_TASKS_PER_NODE' in os.environ

# Check if the Jupyter notebook is running under a SLURM job
if is_slurm_job():
    print("Running on a SLURM-managed cluster.")
    print(f"SLURM_JOB_ID: {os.environ.get('SLURM_JOB_ID')}")
    print(f"SLURM_NODELIST: {os.environ.get('SLURM_NODELIST')}")
else:
    print("Running locally (not on a SLURM-managed cluster).")

Running on a SLURM-managed cluster.
SLURM_JOB_ID: 931602
SLURM_NODELIST: atl1-1-01-002-4-0


# Imports

In [30]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys
sys.path.append('/home/hice1/mbibars3/scratch/vlm-debiasing/VLM-Debiasing-Project/scripts')

In [32]:
%autoreload 2
import extract_audio_features

In [33]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification
import pandas as pd

In [34]:
# Load the labels CSV
csv_path = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/original/labels/detailed_lables.csv"
labels_df = pd.read_csv(csv_path)
labels_df.head()

,Participant,PHQ8_1_NoInterest,PHQ8_2_Depressed,PHQ8_3_Sleep,PHQ8_4_Tired,PHQ8_5_Appetite,PHQ8_6_Failure,PHQ8_7_Concentration,PHQ8_8_Psychomotor,Depression_severity,...,PCL-C_14_Irritability,PCL-C_15_Concentration,PCL-C_16_HyperAlert,PCL-C_17_Jumpy,PTSD_severity,gender,age,Depression_label,PTSD_label,split
0,300,0,0,1,0,1,0,0,0,2,...,2,2,2,1,25.0,male,33,0,0,dev
1,301,0,0,1,1,1,0,0,0,3,...,1,1,1,1,17.0,male,39,0,0,dev
2,302,1,1,0,1,0,1,0,0,4,...,1,1,1,1,28.0,male,25,0,0,train
3,303,0,0,0,0,0,0,0,0,0,...,1,1,1,1,17.0,female,41,0,0,train
4,304,0,1,1,2,2,0,0,0,6,...,1,2,2,1,20.0,female,22,0,0,train


In [35]:
main_dir = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/untarred"
labels_csv = labels_df
cols = ["gender", "split", "PTSD_label","age","PTSD_severity"]

In [36]:
wav_paths_df = extract_audio_features.list_audio_files_paths(main_dir, labels_csv, cols)

100%|██████████| 275/275 [00:00<00:00, 622.04it/s]


In [37]:
wav_paths_df.head()

,gender,split,PTSD_label,age,PTSD_severity,file_path
0,female,train,0,45,22.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
1,male,test,0,69,23.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
2,male,train,0,25,19.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
3,female,train,1,58,67.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
4,male,dev,0,33,39.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...


In [11]:
path_to_save = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic"
file_name = "labels_audio_wavs.csv"
extract_audio_features.save_csv(wav_paths_df, path_to_save, file_name)

# Data Reading

In [43]:
path = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/labels_audio_wavs.csv"
df = pd.read_csv(path)
wav_paths_csv = df["file_path"]

In [13]:
target_path = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/untarred"
# Load model and processor
model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"
processor = AutoFeatureExtractor.from_pretrained(model_name)
model = ASTForAudioClassification.from_pretrained(model_name, output_hidden_states=True)

In [25]:
import numpy as np
arr = np.ones((1200,600))
arr.mean(0)[:,np.newaxis].shape

(600, 1)

In [27]:
# output feature size: (1214, 768)
extract_audio_features.extract_features_AST(wav_paths_csv, target_path, processor, model, pooling=True)

100%|██████████| 275/275 [02:25<00:00,  1.89it/s]


In [44]:
# Add the new column with modified file paths
df["audio"] = df["file_path"].str.replace(".wav", ".npy", regex=False)
df.head()

,gender,split,PTSD_label,age,PTSD_severity,file_path,audio
0,female,train,0,45,22.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
1,male,test,0,69,23.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
2,male,train,0,25,19.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
3,female,train,1,58,67.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
4,male,dev,0,33,39.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...


In [41]:
df["npy_file_path"][0]

'/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/untarred/449_P/449_AUDIO.npy'

In [42]:
import numpy as np
np.load(df["npy_file_path"][0]).shape

(768, 1)

In [45]:
audio_features = df[["split", "PTSD_severity", "PTSD_label",	"gender", "audio"]].copy()
audio_features

,split,PTSD_severity,PTSD_label,gender,audio
0,train,22.0,0,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
1,test,23.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
2,train,19.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
3,train,67.0,1,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
4,dev,39.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
...,...,...,...,...,...
270,test,44.0,1,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
271,test,64.0,1,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
272,test,NaN,1,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
273,train,17.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...


In [46]:
audio_features.to_csv("/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/data_paths.csv")